# Feedback prediction
kaggle: https://www.kaggle.com/competitions/feedback-prize-effectiveness/overview

## Import

In [ ]:
import os
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup

## Data

In [ ]:
# train = pd.read_csv('./data/train.csv')
# test = pd.read_csv('./data/test.csv')
train = pd.read_csv('../input/feedback-prize-effectiveness/train.csv')
test = pd.read_csv('../input/feedback-prize-effectiveness/test.csv')
train.shape, test.shape

In [ ]:
train.head(10)

In [ ]:
test

In [ ]:
type_values = train['discourse_type'].value_counts()
type_values

In [ ]:
target_values = train['discourse_effectiveness'].value_counts()
target_values

## Preprocess

In [ ]:
train_dir = "../input/feedback-prize-effectiveness/train"
test_dir = "../input/feedback-prize-effectiveness/test"
# train_dir = './data/train'
# test_dir = './data/test'
def get_essay(eassy_id, dir):
    essay_dir = os.path.join(dir, f'{eassy_id}.txt')
    essay_txt = open(essay_dir, 'r').read()
    return essay_txt


In [ ]:
train['essay_text'] = train['essay_id'].apply(lambda x: get_essay(x, train_dir))
test['essay_text'] = test['essay_id'].apply(lambda x: get_essay(x, test_dir))
test.head()

In [ ]:
encoder = LabelEncoder()
train['discourse_effectiveness'] = encoder.fit_transform(train['discourse_effectiveness'])
train['discourse_type'] = encoder.fit_transform(train['discourse_type'])
test['discourse_type'] = encoder.fit_transform(test['discourse_type'])

In [ ]:
# train.head()
test.head()

## Dataset

In [ ]:
class EssayDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.discourse_type = df['discourse_type'].values
        self.discourse = df['discourse_text'].values
        self.essay = df['essay_text'].values
        if 'discourse_effectiveness' in self.df:
            self.target = df['discourse_effectiveness'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        discourse_type = self.discourse_type[idx]
        discourse = self.discourse[idx]
        essay = self.essay[idx]
        text = discourse + " " + self.tokenizer.sep_token + " " + essay

        encode_dict = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        ids = encode_dict['input_ids']
        mask = encode_dict['attention_mask']

        ids = ids.squeeze(0)
        mask = mask.squeeze(0)

        if 'discourse_effectiveness' in self.df:
            target = self.target[idx]
            return {"ids" : ids, "mask": mask, "target": target, "dense_feature": discourse_type}
        return {"ids": ids, "mask": mask, "dense_feature": discourse_type}

In [ ]:
model_name = '../input/deberta-v3-base/deberta-v3-base'
# model_name = 'microsoft/deberta-v3-base'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer

In [ ]:
max_len = 512
train_, valid_ = train_test_split(train, test_size=0.2, random_state=42)
train_dataset = EssayDataset(train_, tokenizer, max_len)
# train_dataset = EssayDataset(train, tokenizer, max_len)
valid_dataset = EssayDataset(valid_, tokenizer, max_len)
test_dataset = EssayDataset(test, tokenizer, max_len)

## Model

In [ ]:
class FeedBackModel(nn.Module):
    def __init__(self, model_name):
        super(FeedBackModel, self).__init__()
        self.deberta = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=10, output_attentions=False, output_hidden_states=False)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(10+1, 3)
        
    def forward(self, batch):        
        input_ids, attention_masks, dense_feature = batch['ids'], batch['mask'], batch['dense_feature']
        out = self.deberta(input_ids, attention_mask=attention_masks, output_hidden_states=False)
        output = self.dropout(out.logits)
        dense_feature = dense_feature.reshape((-1, 1))
        output = torch.concat([out.logits, dense_feature], dim=-1)
        output = self.fc(output)
        return output

In [ ]:
# model = FeedBackModel(model_name)
# taloader = DataLoader(valid_dataset, batch_size=2, shuffle=True)
# for batch in teloader:
#     print(batch)
#     print(batch['ids'].shape, batch['mask'].shape, batch['target'].shape)
#     print(batch['target'])
#     print(model(batch))
#     break

In [ ]:
class Classifier(pl.LightningModule):
    def __init__(self, hparams, model):
        super(Classifier, self).__init__()
        self.save_hyperparameters(ignore=['model'])

        self.model = model
        self.batch_size = hparams["batch_size"]
        self.lr = hparams["lr"]
        self.wd = hparams['weight_decay']
        self.steps = hparams['total_steps']


    def forward(self, batch):
        output = self.model(batch)
        return output

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.model.parameters(), lr=self.lr, weight_decay=self.wd)
        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=int(self.steps * 0.1), num_training_steps=self.steps)
        # scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=self.steps)
        scheduler = {"scheduler": scheduler, "interval": "step", "frequency": 1}
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        output = self.forward(batch)
        pred_flat = torch.argmax(output, dim=1).flatten()
        labels_flat = batch['target'].flatten()
        loss = F.cross_entropy(output, labels_flat)
        acc = torch.sum(pred_flat == labels_flat) / len(labels_flat)
        self.log("train_loss", loss, on_epoch=True, on_step=False)
        self.log("train_acc", acc, on_epoch=True, on_step=False)
        return loss

    def validation_step(self, batch, batch_idx):
        output = self.forward(batch)
        pred_flat = torch.argmax(output, dim=1).flatten()
        labels_flat = batch['target'].flatten()
        loss = F.cross_entropy(output, labels_flat)
        acc = torch.sum(pred_flat == labels_flat) / len(labels_flat)
        self.log("val_loss", loss)
        self.log("val_acc", acc)
#         return {"val_loss", loss}

    def train_dataloader(self):
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=2, pin_memory=True)
        return train_loader

    def val_dataloader(self):
        valid_loader = DataLoader(valid_dataset, batch_size=self.batch_size, shuffle=False, num_workers=2, pin_memory=True)
        return valid_loader

    def test_dataloader(self):
        test_loader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=2, pin_memory=True)
        return test_loader

## Training

In [ ]:
pl.seed_everything(42)
hparams = {
    "batch_size": 16,
    "lr": 2e-5,
    "weight_decay": 1e-2,
    "epochs": 1,
}
loader = DataLoader(train_dataset, batch_size=hparams["batch_size"])
hparams["total_steps"] = len(loader) * hparams["epochs"]

In [ ]:
feedback_model = FeedBackModel(model_name)
lightning = Classifier(hparams, feedback_model)

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="./ckpts/",
    mode='min',
    filename='best',
)
# lr_monitor = LearningRateMonitor(logging_interval='step')
trainer = pl.Trainer(
    gpus=1, 
    max_epochs=hparams["epochs"], 
    precision=16, 
    gradient_clip_val=1.0, 
    val_check_interval=0.5,
    callbacks=[checkpoint_callback]
)

In [ ]:
trainer.fit(lightning)

## Inference

In [ ]:
predictions = trainer.predict(dataloaders=lightning.test_dataloader(), ckpt_path='best')

In [ ]:
preds = []
for batch in predictions:
  preds.append(batch)

preds = torch.concat(preds)
preds = preds.type(torch.float32)
preds = F.softmax(preds, dim=1)
# preds.shape
sample = pd.read_csv("../input/feedback-prize-effectiveness/sample_submission.csv")
sample['Adequate'] = preds[:, 0]
sample['Effective'] = preds[:, 1]
sample['Ineffective'] = preds[:, 2]
print(sample.head())
sample.to_csv("submission.csv", index=False)